In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive"

'01.12.15 AM.zip'
'01.12.15 AM.zip (Unzipped Files)'
'02.12.15 AM.zip'
'02.12.15 AM.zip (Unzipped Files)'
'02.12.15 AM.zip (Unzipped Files) (1)'
'04.05.2017 AM.zip'
'04.05.2017 AM.zip (Unzipped Files)'
'04.05.2017 AM.zip (Unzipped Files) (1)'
'04.12.15 AM.zip'
'04.12.15 AM.zip (Unzipped Files)'
'05.04.2016 AM.zip'
'05.04.2016 AM.zip (Unzipped Files)'
 07.09.zip
'07.09.zip (Unzipped Files)'
'07.12.15 AM.zip'
'07.12.15 AM.zip (Unzipped Files)'
'07.12.2016 AM.zip'
'08.12.15 AM.zip'
'08.12.15 AM.zip (Unzipped Files)'
'08.12.15 AM.zip (Unzipped Files) (1)'
'08.12.2016 AM.zip'
'08.12.2016 AM.zip (Unzipped Files)'
'08.12.2016 AM.zip (Unzipped Files) (1)'
'09.04.2016 AM.zip'
'09.04.2016 AM.zip (Unzipped Files)'
 09.09.zip
'09.09.zip (Unzipped Files)'
 11.05.2016_.zip
'11.05.2016_.zip (Unzipped Files)'
 12.09.zip
'12.09.zip (Unzipped Files)'
'13.10.2015 PM.zip'
'13.10.2015 PM.zip (Unzipped Files)'
'14.10.2015 AM.zip'
'14.10.2015 AM.zip (Unzipped Files)'
'14.12.15 AM.zip'
'14.12.15 AM.zip (Unzip

In [0]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,LSTM,CuDNNLSTM,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import matplotlib.pyplot as plt
#drive/My Drive/crypto/BCH-USD.csv

In [4]:
df=pd.read_csv("drive/My Drive/crypto/BCH-USD.csv", names=["time","low",
                                      "high","open","close","volume"],index_col=False)
print(df.head())


         time         low        high        open       close     volume
0  1528968660  871.650024  871.729980  871.650024  871.719971   5.675361
1  1528968720  870.859985  871.719971  871.719971  870.859985  26.856577
2  1528968780  870.099976  871.090027  871.090027  870.099976   1.124300
3  1528968840  868.830017  870.950012  868.830017  870.789978   1.749862
4  1528968900  870.000000  870.000000  870.000000  870.000000   1.680500


In [0]:
SEQ_LEN=60
FUTURE_PERIOD_PREDICT=3
RATIO_TO_PREDICT="LTC-USD"
EPOCHS=10
BATCH_SIZE=64
NAME=f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

In [0]:
def classify(current,future):
    if float(future)>float(current):
        return 1
    else:
        return 0

In [0]:
def preprocess_df(df):
    df = df.drop("future", 1)  

    for col in df.columns:  
        if col != "target":  
            df[col] = df[col].pct_change()  
            df.dropna(inplace=True) 
            df[col] = preprocessing.scale(df[col].values)  

    df.dropna(inplace=True) 
    sequential_data = []  
    prev_days = deque(maxlen=SEQ_LEN) 

    for i in df.values: 
        prev_days.append([n for n in i[:-1]]) 
        if len(prev_days) == SEQ_LEN:  
            sequential_data.append([np.array(prev_days), i[-1]])  

    random.shuffle(sequential_data)  
    
    buys=[]
    sells=[]
    
    for seq,target in sequential_data:
        if target==0:
            sells.append([seq,target])
        elif target==1:
            buys.append([seq,target])
            
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower=min(len(buys), len(sells))
    
    buys=buys[:lower]
    sells=sells[:lower]
    
    sequential_data=buys+sells
    random.shuffle(sequential_data)
    
    X=[]
    Y=[]
    
    for seq,target in sequential_data:
        X.append(seq)
        Y.append(target)
        
    return np.array(X),Y    
    
    

In [8]:
main_df=pd.DataFrame()
ratios=["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]
for ratio in ratios:
    dataset=f"drive/My Drive/crypto/{ratio}.csv"
    df=pd.read_csv(dataset,names=["time","low", "high","open","close","volume"],index_col=False)
    
    df.rename(columns={"close":f"{ratio}_close","volume":f"{ratio}_volume"},inplace=True)
    
    df.set_index("time",inplace=True)
    df=df[[f"{ratio}_close",f"{ratio}_volume"]]
    if len(main_df)==0:
        main_df=df
    else:
        main_df=main_df.join(df)

        
main_df["future"]=main_df[f"{RATIO_TO_PREDICT}_close"].shift\
                                               (-FUTURE_PERIOD_PREDICT)        

main_df["target"]=list(map(classify,main_df\
                    [f"{RATIO_TO_PREDICT}_close"],main_df["future"]))
print(main_df[[f"{RATIO_TO_PREDICT}_close","future","target"]].head(10)) 

times=sorted(main_df.index.values)
#separating the test data
last_5pct=times[-int(0.05*len(times))]
validation_main_df=main_df[(main_df.index>=last_5pct)]
#training data
main_df=main_df[(main_df.index<last_5pct)]


            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0


In [9]:
train_x,train_y=preprocess_df(main_df)
validation_x,validation_y=preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0),train_x.shape}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 69188 validation: 3062
Dont buys: (34594, (69188, 60, 8)), buys: 34594
VALIDATION Dont buys: 1531, buys: 1531


In [0]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
#(timesteps*data_dim )
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs

model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [12]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

filepath = "/content/drive/My Drive/crypto/RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

# Train model
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
)

# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("/content/drive/My Drive/crypto/{}".format(NAME))

Train on 69188 samples, validate on 3062 samples
Epoch 1/10
69188/69188 [==============================] - 75s 1ms/step - loss: 0.6548 - acc: 0.6123 - val_loss: 0.6944 - val_acc: 0.5496
Epoch 2/10
69188/69188 [==============================] - 67s 975us/step - loss: 0.6470 - acc: 0.6214 - val_loss: 0.7040 - val_acc: 0.5519
Epoch 3/10
69188/69188 [==============================] - 68s 986us/step - loss: 0.6362 - acc: 0.6331 - val_loss: 0.7029 - val_acc: 0.5604
Epoch 4/10
69188/69188 [==============================] - 66s 950us/step - loss: 0.6227 - acc: 0.6481 - val_loss: 0.7301 - val_acc: 0.5460
Epoch 5/10
69188/69188 [==============================] - 68s 988us/step - loss: 0.6083 - acc: 0.6642 - val_loss: 0.7575 - val_acc: 0.5457
Epoch 6/10
69188/69188 [==============================] - 67s 965us/step - loss: 0.5939 - acc: 0.6789 - val_loss: 0.7544 - val_acc: 0.5480
Epoch 7/10
69188/69188 [==============================] - 67s 971us/step - loss: 0.5757 - acc: 0.6949 - val_loss: 0.761

In [15]:
my_model=load_model('drive/My Drive/crypto/RNN_Final-02-0.575.model')
score = my_model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6795209110912196
Test accuracy: 0.5721750489097261


In [22]:
pred=my_model.predict(validation_x)
crrct=0
x=[]
res=[]
for i in range(len(pred)):
    if pred[i][0]>=pred[i][1]:
        temp=0
    else:
        temp=1
    
    res.append(temp)
    x.append(i)
    if temp==validation_y[i]:
        crrct=crrct+1
        
print((crrct/len(pred))*100)        
#print(validation_y)

57.21750489875899


In [28]:

y_actu = pd.Series(validation_y, name='Actual')
y_pred = pd.Series(res, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(df_confusion)

Predicted     0     1   All
Actual                     
0.0         671   860  1531
1.0         450  1081  1531
All        1121  1941  3062
